<h2> Building a maultivariable regressor using crime data in New York State in 2013(Source: FBI, UCR.)
We try to predict occurrence of murder in any city using other crime related data</h2>

To reduce the risk of overfitting we will apply following regularization procedures for the linear regression:

- Ridge Regression: where Ordinary Least Squares is modified to also minimize the squared absolute sum of the coefficients (called L2 regularization).
- Lasso Regression: where Ordinary Least Squares is modified to also minimize the absolute sum of the coefficients (called L1 regularization).


## Part 1: Use regular OLS and try to improve it by L1 L2 procedures
## Part 2: Use Logistic Regression and try to improve it by L1 L2 procedures

In [228]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
%matplotlib inline
sns.set_style('white')

In [229]:
# Grab and process the cleaned data
path1 = ("C:/Users/aath/Dropbox/MAEN/Thankful/Data/NY Crime/NY_Crime_2013_cleaned2.csv")

df = pd.read_csv(path1)

In [230]:
df.head()

,City,prop,pop,murd,rob,violent,assul,burg,theft,motor,murd_binary
0,Adams Village,11,1851,0,0,0,0,1,10,0,0
1,Addison Town and Village,49,2568,0,1,2,1,1,47,1,0
2,Afton Village4,1,820,0,0,0,0,0,1,0,0
3,Akron Village,17,2842,0,0,1,1,0,17,0,0
4,Albany4,3888,98595,8,237,802,503,683,3083,122,1


In [231]:
# Prepare the data
# Remove New York City as it is the largest city in NY State and does distort visualization
df = df[df.City != 'New York']

# Murder = murd is our dependent binary variable already represented by murd_binayr. 
# City is not 
df = df.drop('murd',1)
df = df.drop('City',1)

In [232]:
df.isnull().sum()

prop           0
pop            0
rob            0
violent        0
assul          0
burg           0
theft          0
motor          0
murd_binary    0
dtype: int64

In [233]:
import numpy as np
trainsize = int(df.shape[0] / 2)
chosen_idx1 = np.random.choice(df.shape[0], replace=False, size=trainsize)
chosen_idx2 = np.random.choice(df.shape[0], replace=False, size=trainsize)

In [234]:
#chosen_idx2 = chosen_idx1[~chosen_idx1.index.isin(chosen_idx1.index)]

In [235]:
# Define the training and test sizes.
df.sample(frac=1) # Shuffle the data
trainsize = int(df.shape[0] / 2)
df_test = df.iloc[trainsize:, :].copy()
df_train = df.iloc[:trainsize, :].copy()

# Set up the regression model to predict defaults using all other
# variables as features.
regr1 = linear_model.LinearRegression()
Y_train = df_train['murd_binary'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['murd_binary'])]
regr1.fit(X_train, Y_train)
print('\nR-squared simple model:')
print(regr1.score(X_train, Y_train))


R-squared simple model:
0.370176275746


- This model explains 37% of variability of the response data around the mean.
- R-squared = Explained variation / Total variation

In [236]:
#Store the parameter estimates.
origparams = np.append(regr1.coef_, regr1.intercept_)
origparams

array([ -1.42384974e-03,   5.50090802e-06,  -6.38857371e-03,
         9.52125573e-03,  -9.31481865e-03,  -2.31074785e-04,
         1.72793299e-03,  -2.92070794e-03,   1.97542960e-02])

In [237]:
# Make new features to capture potential quadratic
# between the features.

df_train['prop2'] = (df_train['prop'] + 100) ** 2
df_train['pop2'] = (df_train['pop'] + 100) ** 2
df_train['rob2'] = (df_train['rob'] + 100) ** 2
df_train['violent2'] = (df_train['violent'] + 100) ** 2
df_train['assul2'] = (df_train['assul'] + 100) ** 2
df_train['burg2'] = (df_train['burg'] + 100) ** 2
df_train['theft2'] = (df_train['theft'] + 100) ** 2
df_train['mortor2'] = (df_train['motor'] + 100) ** 2

In [238]:
# Re-run the model with the new features.
regrBig = linear_model.LinearRegression()
X_train2 = df_train.loc[:, ~(df_train.columns).isin(['murd_binary'])]
regrBig.fit(X_train2, Y_train)
print('\nR-squared complex model:')
print(regrBig.score(X_train2, Y_train))


R-squared complex model:
0.528070657814


- R-squared increases by added features

In [239]:
# Store the new parameter estimates for the same features.
newparams = np.append(
    regrBig.coef_[0,0:(len(origparams)-1)],
    regrBig.intercept_)

print('\nParameter Estimates for the same predictors for the small model '
      'and large model:')
compare = np.column_stack((origparams, newparams))
prettycompare = np.array2string(
    compare,
    formatter={'float_kind':'{0:.3f}'.format})
print(prettycompare)


Parameter Estimates for the same predictors for the small model and large model:
[[-0.001 -0.016]
 [0.000 0.000]
 [-0.006 0.030]
 [0.010 0.009]
 [-0.009 -0.010]
 [-0.000 0.013]
 [0.002 0.016]
 [-0.003 -0.045]
 [0.020 -1.935]]


- After adding extra features, the magnitute and sign of some of coefficients changed. This can be a sign of overfitting and of having too many correlated dimensions.

In [240]:
# Test the simpler model with smaller coefficients.
Y_test = df_test['murd_binary'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_test.columns).isin(['murd_binary'])]
print('\nR-squared simple model:')
print(regr1.score(X_test, Y_test))

# Test the more complex model with larger coefficients.

df_test['prop2'] = (df_test['prop'] + 100) ** 2
df_test['pop2'] = (df_test['pop'] + 100) ** 2
df_test['rob2'] = (df_test['rob'] + 100) ** 2
df_test['violent2'] = (df_test['violent'] + 100) ** 2
df_test['assul2'] = (df_test['assul'] + 100) ** 2
df_test['burg2'] = (df_test['burg'] + 100) ** 2
df_test['theft2'] = (df_test['theft'] + 100) ** 2
df_test['mortor2'] = (df_test['motor'] + 100) ** 2

# Re-run the model with the new features.
X_test2 = df_test.loc[:, ~(df_test.columns).isin(['murd_binary'])]
print('\nR-squared complex model:')
print(regrBig.score(X_test2, Y_test))


R-squared simple model:
-468.526557014

R-squared complex model:
-13997302.8083


- Testing the model on remaining data set result in a very poor outcome. A negative R-squared indicates that the fit of the model is even worse than a horizontal line. 

# Next, we try to improve the model by using Ridge procedure. 

In [241]:
# To address the overfitting problem we can neglect the requirement of an unbiased parameter estimator 
# and instead use a biased estimator, which may have smaller variance.
# One of such biased estimators is the Ridge method. 
# Below we will fit a ridge regression model where alpha is the regularization parameter (usually called lambda). 
# As alpha gets larger, parameter shrinkage grows more pronounced. Note that by convention, the intercept is not regularized. 

ridgeregr = linear_model.Ridge(alpha=20, fit_intercept=False) 
ridgeregr.fit(X_train, Y_train)
print(ridgeregr.score(X_train, Y_train))
origparams = ridgeregr.coef_[0]
print(origparams)

ridgeregrBig = linear_model.Ridge(alpha=20, fit_intercept=False)
ridgeregrBig.fit(X_train2, Y_train)
print(ridgeregrBig.score(X_train2, Y_train))
newparams = ridgeregrBig.coef_[0, 0:len(origparams)]

print('\nParameter Estimates for the same predictors for the small model'
      ' and large model:')
compare = np.column_stack((origparams, newparams))
prettycompare = np.array2string(
    compare,
    formatter={'float_kind':'{0:.3f}'.format})
print(prettycompare)

0.367976300523
[ -1.35481010e-03   6.50818947e-06  -6.06347569e-03   8.63844577e-03
  -8.20213564e-03  -1.62541232e-04   1.62321631e-03  -2.81548517e-03]
0.519280758028

Parameter Estimates for the same predictors for the small model and large model:
[[-0.001 -0.007]
 [0.000 0.000]
 [-0.006 0.032]
 [0.009 -0.006]
 [-0.008 0.015]
 [-0.000 0.002]
 [0.002 0.007]
 [-0.003 -0.016]]


In [242]:
print('\nR-squared simple model:')
print(ridgeregr.score(X_test, Y_test))
print('\nR-squared complex model:')
print(ridgeregrBig.score(X_test2, Y_test))


R-squared simple model:
-504.075019864

R-squared complex model:
-16245882.5219


- Risdge regression using regularization parameter λ = 20 make the model worse.

# Next, we try to improve the model by using Lasso procedure. 

Following we will use Lasso Regression which is a close cousin of Ridge Regression, in which absolute values of coefficients are minimized rather than the square of values. This method will help to eliminate some insignificant variables. Lasso regression is generally used when there is a very large number of variables, since Lasso automatically does the variables selection.



In [243]:
# Small number of parameters.
lass = linear_model.Lasso(alpha=.35)
lassfit = lass.fit(X_train, Y_train)
print('R² for the model with few features:')
print(lass.score(X_train, Y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

# Large number of parameters.
lassBig = linear_model.Lasso(alpha=.35)
lassBig.fit(X_train2, Y_train)
print('\nR² for the model with many features:')
print(lassBig.score(X_train2, Y_train))
origparams = np.append(lassBig.coef_, lassBig.intercept_)
print('\nParameter estimates for the model with many features:')
print(origparams)

R² for the model with few features:
0.348956285342

Parameter estimates for the model with few features:
[ -0.00000000e+00   4.77910101e-06  -0.00000000e+00   8.73183121e-05
   0.00000000e+00  -9.72696778e-04   3.33892171e-04  -0.00000000e+00
   1.51999064e-02]

R² for the model with many features:
0.46285409656

Parameter estimates for the model with many features:
[  4.50473213e-04  -2.60139952e-09   0.00000000e+00   1.15178047e-03
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -9.07274967e-08   2.94768075e-11  -7.03863650e-07  -1.93179870e-07
  -3.28346609e-07   5.55158235e-07  -5.68330024e-08   6.01014665e-06
  -5.94734578e-02]


C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [244]:
# Checking predictive power using the test set:
print('\nR-squared simple model:')
print(lass.score(X_test, Y_test))
print('\nR-squared complex model:')
print(lassBig.score(X_test2, Y_test))


R-squared simple model:
-194.185555919

R-squared complex model:
-19706.5628076


- Using the Lasso method the model improves considerably compared to previous models. However, because of negative R-squared we can conclude that this model, with its constraints, fits the data really poorly.
- We can also see that the Lasso method is able to make a selection of variables. Overall we can say that both Lasso and Ridge balance the trade-off bias-variance with the choice of lambda. On this dataset the Lasso performed better and apparently it was because of many of predators were not actually tied to the response variable. And to test this one should use cross-validation and also try to do an stepwise regression. Here we started with all variables and tried to reduce or shrink them. Another approach is to go forward stepwise by starting with an empty model and choose the variables with most significant association. In each step we add variable that can provide statistically significant association with the dependent variable. By using cross-validation and pipeline module we should be able to find and select the best variables in a backward and forward stepwise fashon.

## Part 2: Let's try fitting a binary logistic model using statsmodels to see if we can improve R-squared

In [245]:
# Declare predictors.
X_statsmod = X_train2.copy()

# The Statsmodels formulation requires a column with constant value 1 that
# will act as the intercept.
X_statsmod['intercept'] = 1 

# Declare and fit the model.
logit = sm.Logit(Y_train, X_statsmod)
result = logit.fit()

# Lots of information about the model and its coefficients, but the
# accuracy rate for predictions is missing.
print(result.summary())

         Current function value: 0.165594
         Iterations: 35


C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)


                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  184
Model:                          Logit   Df Residuals:                      168
Method:                           MLE   Df Model:                           15
Date:                Thu, 14 Jun 2018   Pseudo R-squ.:                  0.5476
Time:                        14:18:01   Log-Likelihood:                -30.469
converged:                      False   LL-Null:                       -67.355
                                        LLR p-value:                 9.434e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
prop          -0.5265        nan        nan        nan         nan         nan
pop            0.0001   7.95e-05      1.423      0.155   -4.27e-05       0.000
rob            1.4096      1.097      1.286      0.1

C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [246]:
# Prepare the output arrays for tabular form so can use table function 
Y_train.shape, pred_y_statsmod.shape

((184, 1), (184,))

In [247]:
Y_train_1D = Y_train.ravel()  # We need to convert Y Train 2D to 1D

In [248]:
# Calculate accuracy. First, get probability that each row will be admitted.
pred_statsmod = result.predict(X_statsmod)

# Code admission as 1 if probability is greater than .5.
pred_y_statsmod = np.where(pred_statsmod < .5, 0, 1)

# Accuracy table.
table = pd.crosstab(Y_train_1D, pred_y_statsmod)

print('\n Accuracy by admission status')
print(table)
print('\n Percentage accuracy')
print((table.iloc[0,0] + table.iloc[1,1]) / (table.sum().sum()))


 Accuracy by admission status
col_0    0   1
row_0         
0      160   2
1        8  14

 Percentage accuracy
0.945652173913


### The accuracy of the model is very high but becasue of class imbalance the result is not useful.

In [249]:
Y_train.shape, X_test2.shape, X_train2.shape

((184, 1), (184, 16), (184, 16))

### Below we do the same using SKlearn LogisticRegression method and this time try to manipualte  "penalty" argument of L2

In [261]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
    
# Create a classifier object
lr = LogisticRegressionCV(penalty='l2', Cs=10, random_state=0, n_jobs=-1)

params = {'Cs':[10, 100, 500]}


# instantiate a gridsearch class
grid = GridSearchCV(lr, params)

# Train model
grid.fit(X_train2, Y_train)

C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vecto

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=0,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'Cs': [10, 100, 500]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [264]:
# check the best params
grid.best_params_

{'Cs': 10}

## We tried multiple penalty parameters and it seems C = 10 is the best one when using L2.
## We cannot run the same LogisticRegressionCV for L1 as this SKlearn module does not suupport it. But we can try couple of C manually.  

In [269]:
# Create a classifier object
logistic_regression = LogisticRegression(C=10, penalty='l1')

# Train model
model1 = logistic_regression.fit(X_train2, Y_train)
# print('\nR-squared complex model:')
print(model1.score(X_test2, Y_test))

0.880434782609


C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [270]:
# Create a classifier object
logistic_regression = LogisticRegression(C=1000, penalty='l1')

# Train model
model1 = logistic_regression.fit(X_train2, Y_train)
# print('\nR-squared complex model:')
print(model1.score(X_test2, Y_test))

C:\Users\aath\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.880434782609


## It seems L1 method for this data set is not sensitive with different level of C.